# How to Get to Everyone

Last meeting, SUCO hosted a panel with various GT faculty. During the panel, there was a little discussion on how you can collect attendance.

One way to do it is one by one, passing the attendance sheet around the room, one person at a time.

Person -> Person -> Person -> Person -> Person -> ... -> Person

This takes linear time, or O(n). This algorithm also makes use of people terribly, with between two to one people active at any given time.

Person -> Person | Person - idle | Person - idle | Person -idle | ... | Person - idle

Another suggestion on how to do attendance was by row. Each row would have an attendance sheet, passing it from one end to another, then the last person of each row will pass the attendance sheet forward, accumulating all the sheets in the first row for the SUCO president to tally.

Person -> Person -> Person -> Person V

Person -> Person -> Person -> Person V

Person -> Person -> Person -> Person V

Person -> Person -> Person -> Person V

Person -> Person -> Person -> Person V

Person -> Person -> Person -> Person V

                                    SUCO president

This algorithm takes two steps. First each row gathers the names of the row. Then the last person in the row passes the sheet forward. 

Gathering the names of the row can be done in parallel. This will take O(__total_people__ / __total_rows__) assuming each row has an equal number of people. If not, this step is bound by the row with the most people. Let us also assume people in the same row work faster together (as if they are one processor), so there is no communication overhead here.

The next step takes O(__total_rows__). But, let's pretend people between rows are not as good at getting each other's attention so it takes a second for each pass to occur. This adds an additional part to the runtime that is tied to communication. Let's say O(__time_taken_to_get_someone_else's_attention__) * __total_rows__.**

And so, the total time it takes to run this algorithm is O(__total_people__ / __total_rows__) + O(__total_rows__) + O(__time_taken_to_get_someone_else's_attention__) * __total_rows__.**

**Further details about communication and HPC considerations for algorithms can be found on the internet or in future notebooks

This algorithm definitely had more people involved at once compared to the first algorithm. But, there are many spots in the algorithm that could go wrong, like having too many people in one of the rows(load balancing issue), or having too many rows(communication takes over). These factors will slow the algorithm down (as well as any other parallel algorithm). 

But, there is another way to do attendance. Sarkar mentioned it briefly, but there is a way you can whittle the algorithm runtime to O(log(n)). He also mentioned that you'd have to rearrange yourselves into trees, and that is true, this algorithm will create a communication pattern that looks like a binary tree.

A binary tree* looks like this:

![Binary tree for visual purposes](binary-tree.png)

It is only here to give a first look at what's going on in this algorithm we're discussing. We're not going to chat about how to reverse a binary tree.

*If you haven't heard of binary trees before, here's a comprehensive [guide](http://cslibrary.stanford.edu/110/BinaryTrees.html). This is way more than you need to know relative to what's being explained in this notebook, but it will help you in the future when it comes to devising algorithms. You'll also get a first look at this in CS 1332, or Data Structures and Algorithms.

The main thing to note from this visual is that each level has double the nodes than the previous one. This is exponential growth, specifically 2^n. Bad as an algorithm time, great for scaling parallel algorithms.


Now, in our algorithm, we're going to have half the room write their name on individual sheets of paper. 

Then, in pairs, they will give their sheet of papers to someone who doesn't have one. They will merge the sheets of paper together, write their name on it, and in pairs, hand their sheets over to someone who still doesn't have one. 

Rinse and repeat until there is only one person with the merged sheet of paper. Realistically, paper can't be merged and this person will just end up with a pile of paper. (if you are trying to visualize this, the binary tree above is what it should look like).

This algorithm still suffers from the case where eventually most people aren't doing anything (there are alternate algorithms that keep everyone occupied. To be discussed), but it takes only __log(n)__ rounds for the last person to receive the completed attendance sheet. Communication rounds scale with number of people rather than number of rows from the previous algorithm. We are able to consider each person their own processor and still have a faster algorithm than the row algorithm (discussed in a later notebook).

### __The Details__

The last section we gave a high level description of an algorithm. Now let's look into the details.

We mentioned that people would be handing attendance sheets to other people such that each person who receives the sheets has two sheets. How do we coordinate that? For all we know, someone could receive three sheets of paper, or someone who should have received a paper is empty-handed. People who've already received a paper might also get involved in later rounds. It's very easy for this to devolve into chaos without any agreement on who receives from who.

We could have everyone arrange themselves into a binary tree structure. But in a lecture hall that will result in a lot shuffling. Sometimes you don't have the luxury of shuffling people around.

Before we go into routing, try to think about how _you would coordinate people in receiving and sending papers_.

Things to consider:
- People don't have to be one and done. They can take part in later rounds.
- People can recieve as many papers as they want, but do note that this might result in a bottleneck.
- People can send as many papers as they want, but a person doing 4 sends will take longer than someone doing 2 sends
- People can receive and send at the same time (or if you're doing rounds, same round).
- The end goal here is for one person to end up with all the papers, but, we're not opposed to everyone receiving all the papers.

#### __Bit-fixing routing__

Bit-fixing routing is quite the clever way of routing without any need for an external source to determine routes. With algorithms like the one we discussed, it enables a no-collisions way to work in tandem. (As a standalone means of routing there are flaws, but they are not present in what we're using it for). 

#### __Binary Numbers__

Before we get into it, you're going to need to know what binary numbers are. They are key to this routing method. If you do, skip to the next section. 

You've likely been working in a base 10(aka. decimal) number system all your life. And you probably would have stayed in that system had you not decided to touch computers. But you did, so now we're going to give you an overview of binary numbers, or a base 2 number system.

__Number Bases__

The base 10 system has 10 unique digits: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.

Every single number in the decimal system is represented by some combination of these numbers. Sometimes there are multiple digits stuck together to represent numbers in the 10s or 1000s. 

Ex. 5, 39, 123

You know what the above values are cause you've been exposed to this system since daycare. But why are these values the values that they are? Why is the 1 in 123 representative of 100 and not 1? 

__Translating numbers to base 10__

Let's look at a number. What value do you think this is?

11011010

Without any knowledge of how bases work, you might interpret this as eleven million eleven thousand ten assuming it's base 10. But what if it wasn't base 10? This number could literally represent infinite other values. There's a base out there that interprets this as 1(base 11011010). In the computer world you can no longer assume every number you see is meant to be interpreted in base 10. 

What's truly fascinating is that depending on the base you interpret this number, you get different values. This is the goal of this section, to acquaint you with how to interpret numbers in other bases, as computer science will expose you to other base systems such as base 2 (binary), base 8 (octal), and base 16 (hexadecimal).

Let's take the number and interpret it in base 10, 3, and 2.

Base 10:

11011010 -> eleven million eleven thousand ten.

Why do we know this? Well if we number the position of each digit, starting with 0 for the right most position, and expand it as so:

$$1 * 10^7 + 1 * 10^6 + 0 * 10^5 + 1 * 10^4 + 1 * 10^3 + 0 * 10^2 + 1 * 10^1 + 0 * 10^0 = 11011010$$

$$10,000,000 + 1,000,000 + 0 + 10,000 + 1,000 + 0 + 10 + 0 = 11011010$$

This doesn't really demonstrate the effect of bases as we default to interpreting things in base 10, so let's interpret the number in the base 3 system and translate it to base 10 so that we can understand its value:

$$1 * 3^7 + 1 * 3^6 + 0 * 3^5 + 1 * 3^4 + 1 * 3^3 + 0 * 3^2 + 1 * 3^1 + 0 * 3^0 = 3027$$

$$2187 + 729 + 0 + 81 + 9 + 0 + 3 + 0 = 3027$$

Now let's interpret it as base 2 and translate it to base 10:

$$1 * 2^7 + 1 * 2^6 + 0 * 2^5 + 1 * 2^4 + 1 * 2^3 + 0 * 2^2 + 1 * 2^1 + 0 * 2^0 = 218$$

$$128 + 64 + 0 + 16 + 8 + 0 + 2 + 0 = 218$$

If you want to translate a number to a non-base 10 system, the above concept stays the same. However, you must take care to translate the resulting values into the base you are translating to. Addition must also occur in the target base. To illustrate this, lets translate 11011010 from base 2 to base 3.

$$1 * 2^7 + 1 * 2^6 + 0 * 2^5 + 1 * 2^4 + 1 * 2^3 + 0 * 2^2 + 1 * 2^1 + 0 * 2^0$$

$$11202 + 2101 + 0 + 121 + 22 + 0 + 2 + 0$$

$$22002$$

If you translate this from base 3 to base 10, you'll get 218, which was the value we got from translating from base 2 to base 10.

__Translating numbers to base 2__

In the previous example we only translated from lower bases to higher bases. What if we want to go from base 10 to base 2?

Using the previous example, let's translate 218 from base 10 to base 2.

There are multiple ways to calculate it, but we will calculate it as so:

218 / 2 = 109 R0

109 / 2 = 54 R1

54 / 2 = 27 R0

27 / 2 = 13 R1

13 / 2 = 6 R1

6 / 2 = 3 R0

3 / 2 = 1 R1

1 / 2 = 0 R1

To sum the above, you divide the number by 2, record the remainder, then divide by 2 again and repeat this until you reach 0. Then, take the remainders, starting with the one for 0 and moving upwards, and line them in a row from left to right.

With the above, you get 11011010, which was the number we translated 218 from base 2 to base 10.

If you want to apply this to other bases, divide by the other base rather than 2.

Now that we've gone over binary numbers, let us get to bit-fixing routing.

#### __Bit-Fixing Cont.__

Before we get to bit-fixing, let's give every person(processor) an unique number. This identifier is referred to as the rank and is what we'll use to denote who communicates with who. 

For our example we'll have 8 people involved, and assigned them ranks as so:

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |

These numbers are in base 10, but for bit-fixing, we'll translate them into binary:

| 000 | 001 | 010 | 011 | 100 | 101 | 110 | 111 |

You might notice that for ranks 0, 1, 2, and 3, we've left the left-most zeros in. This is all part of the plan.

Now, the algorithm. We're going to demonstrate bit-fixing with a packet that's to go from 000 to 111.

How will we get from 000 to 111? Numerically speaking, we flip every bit from 0 to 1. 

So, if we take it one step at a time:

000 -> 100

100 -> 110

110 -> 111

And there you go, that's the path the packet. We're going to send it from 000 -> 100 -> 110 -> 111 (0 -> 4 -> 6 -> 7).

Changing the order of flipping simply reshuffles the order of these ranks and is also valid.

What about from 101 to 001? In this case, only one bit differs, so we only need to flip that bit to get the rank to match.

101 -> 001

And that's the path. This packet will go from 101 -> 001 (5 -> 1).

The steps for bit-fixing:

- Determine the source and destination.

- Note which bits differ in the source and destination ranks.

- For each different position, flip the bit. That is now either an intermediate destination or destination. Flip bits until the rank you're flipping matches the destination rank.

__Exercises__

What's the bit-fixing path for a packet that wants to go from rank 120 to 36?

#### __Applying it to our algorithm for attendance__

So, what's bit-fixing got to do with attendance? Well, if you don't care about destination (ie just want to send stuff around), you can use bit-fixing routing to coordinate processors such that after log2(n) rounds, everyone will have indirectly communicated with everyone.

By indirectly communicated we mean that if a processor 0 did communication with processor 3 earlier, and is now communicating with you, you've indirectly communicated with processor 3.

3 -> 0 -> you

What we're going to do is have everyone flip their bits. One bit at a time, all at once. It will appear to devolve into chaos, but rest assured, it will make a pretty picture.

For uniformity, we will go from left to right, like we did when we went from 000 to 111.

If we have 8 people to do attendance for, then the first set of bit flips is as so:

| 000 -> 100 | 001 -> 101 | 010 -> 110 | 011 -> 111 | 100 -> 000 | 101 -> 001 | 110 -> 010 | 111 -> 011 |

So the first round the papers move around as so:

| 0 -> 4 | 1 -> 5 | 2 -> 6 | 3 -> 7 | 4 -> 0 | 5 -> 1 | 6 -> 2 | 7 -> 3 |

We're going to have each person effectively merge together the papers, so that each person has the names of two people, their own and the person who sent them a paper.

Now we're going to flip the second bit of each rank to determine the next destination:

| 100 -> 110 | 101 -> 111 | 110 -> 100 | 111 -> 101 | 000 -> 010 | 001 -> 011 | 010 -> 000 | 011 -> 001 |

| 4 -> 6 | 5 -> 7 | 6 -> 4 | 7 -> 5 | 0 -> 2 | 1 -> 3 | 2 -> 0 | 3 -> 1 |

We merge the papers together again, and now each person has the names of four people.

We flip one last time:

| 110 -> 111 | 111 -> 110 | 100 -> 101 | 101 -> 100 | 010 -> 011 | 011 -> 010 | 000 -> 001 | 001 -> 000 |

| 6 -> 7 | 7 -> 6 | 4 -> 5 | 5 -> 4 | 2 -> 3 | 3 -> 2 | 0 -> 1 | 1 -> 0 |

And once we merge the papers together, each person has everyone elses names. If we wanted to, we could make it so that with each round, half the people (lower or higher half) would get rid of their papers or stopped participating in the rounds. This way only one person ends with with all the names. And we would save a lot more paper.


__What is this called?__

The routing method is called bit-fixing, but when doing the above, you might here the term __hypercubic permutation__ thrown around (especially in the Intro to HPC class). From here on out, I will refer to it as __hypercubic permutations__  as well.

__Visualizing the above__

The previous section threw a lot of random numbers at you, but if you map out the connections, you get this graph:

![butterfly-network](butterfly-network.png)

Ignore the straight lines. I could not find the image I was looking for, so I had to settle for a [butterfly network](https://en.wikipedia.org/wiki/Butterfly_network)

#### __Other algorithms that use this sort of permutation__

prefix sum
ffts
broadcast, gather, the comms

### __Try applying it to these problems__

Implement the broadcast exercise in the parallelpy.ipynb with hypercubic permutations.



### __What's so special about bit-fixing?__

hypercubes.